<a href="https://colab.research.google.com/github/mayafetzer/MetalLeaching/blob/main/MetalExtractionMLModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Full pipeline: training, tuning, CV plots, SHAP/importance, Streamlit app, export predictions
!pip install --quiet pandas numpy scikit-learn matplotlib seaborn openpyxl shap streamlit xlrd

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from google.colab import files

from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import shap

# -------------------------
# 1) Upload and prepare data
# -------------------------
print("Upload your Excel file (ReducingAgentData.xlsx):")
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

df_raw = pd.read_excel(file_path, header=None)

# Fix headers (drop first grouped row and promote second row)
df = df_raw.drop(index=0).reset_index(drop=True)
raw_header = df.iloc[0].astype(str)
clean_header = (
    raw_header.str.replace(r"^\s*0\s*", "", regex=True)
              .str.replace("\n", " ", regex=False)
              .str.replace("\r", " ", regex=False)
              .str.strip()
              .str.replace("  ", " ")
)
df.columns = clean_header
df = df.drop(index=0).reset_index(drop=True)

print("Columns:", list(df.columns))

# Define columns (as you specified)
categorical_cols = ["Leaching agent", "Type of reducing agent"]
numeric_cols = [
    "Li in feed %",
    "Co in feed %",
    "Mn in feed %",
    "Ni in feed %",
    "Concentration, M",
    "Concentration %",
    "Time,min",
    "Temperature, C"
]
target_cols = ["Li", "Co", "Mn", "Ni"]

# Ensure numeric conversion
for c in numeric_cols + target_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Drop rows missing any target
df = df.dropna(subset=target_cols).reset_index(drop=True)
print("Dataset after dropping rows without all targets:", df.shape)

# -------------------------
# 2) EDA heatmap
# -------------------------
plt.figure(figsize=(12,8))
corr = df[numeric_cols + target_cols].corr()
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.savefig("heatmap_numeric_targets.png", dpi=150)
plt.show()

# -------------------------
# 3) Preprocessors
# -------------------------
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor_withcat = ColumnTransformer([
    ("num", numeric_pipeline, numeric_cols),
    ("cat", categorical_pipeline, categorical_cols)
])

preprocessor_nocat = ColumnTransformer([
    ("num", numeric_pipeline, numeric_cols)
])

# -------------------------
# 4) Model candidates & param grids (for RandomizedSearch)
# -------------------------
candidates = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

param_distributions = {
    "LinearRegression": {
        # very few params to tune for quickness
        # LinearRegression has no many hyperparams; we'll vary fit_intercept
        "model__fit_intercept": [True, False]
    },
    "Ridge": {
        "model__alpha": [0.001, 0.01, 0.1, 1, 10, 100],
        "model__solver": ["auto"]
    },
    "RandomForest": {
        "model__n_estimators": [100, 200, 400],
        "model__max_depth": [3, 5, 10, None],
        "model__min_samples_split": [2, 5, 10],
        "model__max_features": ["auto", "sqrt"]
    },
    "GradientBoosting": {
        "model__n_estimators": [100, 200, 400],
        "model__learning_rate": [0.01, 0.05, 0.1],
        "model__max_depth": [2, 3, 4]
    }
}

# RandomizedSearch settings
n_iter_search = 20
cv_folds = 5
random_state = 42

# Create output dirs
os.makedirs("models", exist_ok=True)
os.makedirs("cv_plots", exist_ok=True)
os.makedirs("importance_plots", exist_ok=True)

# -------------------------
# 5) Helper functions
# -------------------------
from sklearn.model_selection import cross_val_score

def tune_and_evaluate(pipe, param_dist, X_train, y_train):
    # Randomized search
    rnd = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=n_iter_search,
                             scoring="r2", cv=cv_folds, n_jobs=-1, random_state=random_state, verbose=0)
    rnd.fit(X_train, y_train)
    return rnd

def plot_cv_r2(model, X, y, title, outpath):
    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=random_state)
    scores = cross_val_score(model, X, y, scoring="r2", cv=kf, n_jobs=-1)
    plt.figure(figsize=(7,4))
    plt.plot(range(1, cv_folds+1), scores, marker='o', linestyle='-')
    plt.title(title)
    plt.xlabel("Fold")
    plt.ylabel("R²")
    plt.ylim(-1, 1)
    plt.grid(True)
    plt.savefig(outpath, dpi=150, bbox_inches='tight')
    plt.show()
    return scores

def compute_and_plot_importance(model, preprocessor, X, y, outpath, top_n=20, model_name="model"):
    """
    If tree-based model -> SHAP TreeExplainer (fast).
    Else -> permutation importance.
    """
    # Get preprocessed X (2d array) and feature names
    X_proc = preprocessor.transform(X)
    try:
        feature_names = preprocessor.get_feature_names_out()
    except:
        # fallback
        # numeric names + ohe feature names
        num_names = numeric_cols
        cat_transformer = preprocessor.named_transformers_['cat'] if 'cat' in preprocessor.named_transformers_ else None
        cat_names = []
        if cat_transformer is not None:
            try:
                ohe = cat_transformer.named_steps['onehot']
                cat_names = list(ohe.get_feature_names_out(categorical_cols))
            except:
                cat_names = []
        feature_names = list(num_names) + list(cat_names)

    # If underlying estimator is tree-based, try SHAP TreeExplainer
    base_est = model.named_steps['model']
    try:
        if isinstance(base_est, (RandomForestRegressor, GradientBoostingRegressor)):
            explainer = shap.TreeExplainer(base_est)
            shap_values = explainer.shap_values(X_proc)
            # shap_values will be 2D array (n_samples, n_features)
            shap_abs = np.abs(shap_values).mean(axis=0)
            imp_df = pd.DataFrame({"feature": feature_names, "importance": shap_abs})
            imp_df = imp_df.sort_values("importance", ascending=False).head(top_n)
            # Plot
            plt.figure(figsize=(8, max(4, 0.25*len(imp_df))))
            sns.barplot(x="importance", y="feature", data=imp_df)
            plt.title(f"SHAP mean(|value|) feature importance ({model_name})")
            plt.tight_layout()
            plt.savefig(outpath, dpi=150, bbox_inches='tight')
            plt.show()
            return imp_df
    except Exception as e:
        print("SHAP failed or not applicable:", e)

    # Fallback: permutation importance (slower)
    print("Using permutation importance (fallback). This may take some time.")
    r = permutation_importance(model, X, y, n_repeats=10, random_state=random_state, n_jobs=-1)
    importances = r.importances_mean
    imp_df = pd.DataFrame({"feature": feature_names, "importance": importances})
    imp_df = imp_df.sort_values("importance", ascending=False).head(top_n)
    plt.figure(figsize=(8, max(4, 0.25*len(imp_df))))
    sns.barplot(x="importance", y="feature", data=imp_df)
    plt.title(f"Permutation importance ({model_name})")
    plt.tight_layout()
    plt.savefig(outpath, dpi=150, bbox_inches='tight')
    plt.show()
    return imp_df

# -------------------------
# 6) Loop over targets and both pipelines
# -------------------------
summary_rows = []

for use_cat, preprocessor, label in [
    (True, preprocessor_withcat, "withcat"),
    (False, preprocessor_nocat, "nocat")
]:
    print("\n\n==============================")
    print("Pipeline:", label)
    print("==============================")

    # Select X for pipeline
    X_all = df[numeric_cols + (categorical_cols if use_cat else [])]

    for target in target_cols:
        print("\n----------------------------------------------")
        print(f"TARGET: {target}  |  PIPELINE: {label}")
        print("----------------------------------------------")

        y_all = df[target]

        # train/test split (for tuning)
        X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=random_state)

        # Evaluate each base candidate and record simple baseline
        base_results = []
        for name, estimator in candidates.items():
            pipe = Pipeline([("preprocessing", preprocessor), ("model", estimator)])
            pipe.fit(X_train, y_train)
            preds = pipe.predict(X_test)
            rmse = mean_squared_error(y_test, preds)
            r2 = r2_score(y_test, preds)
            base_results.append((name, rmse, r2))
            print(f"[BASE] {name:18s} RMSE={rmse:.4f}  R2={r2:.4f}")

        # pick winner (based on RMSE)
        base_results.sort(key=lambda x: x[1])
        best_base_name = base_results[0][0]
        print(f"\nInitial best model (by RMSE) for {target} ({label}): {best_base_name}")

        # Prepare param dist keyed by best_base_name
        param_dist = param_distributions.get(best_base_name, {})
        pipe_for_tune = Pipeline([("preprocessing", preprocessor), ("model", candidates[best_base_name])])

        # Attach parameter prefixes to param_dist keys are already prefixed with model__
        # Run RandomizedSearchCV
        if len(param_dist) > 0:
            print("Starting RandomizedSearchCV (this may take a while)...")
            rnd = RandomizedSearchCV(pipe_for_tune, param_distributions=param_dist, n_iter=n_iter_search,
                                     scoring="r2", cv=cv_folds, n_jobs=-1, random_state=random_state, verbose=1)
            rnd.fit(X_train, y_train)
            tuned = rnd.best_estimator_
            print("Tuning complete. Best params:", rnd.best_params_, "Best CV R2:", rnd.best_score_)
        else:
            print("No param grid for this model; skipping tuning.")
            tuned = pipe_for_tune
            tuned.fit(X_train, y_train)

        # Evaluate tuned model on held-out test
        preds_test = tuned.predict(X_test)
        test_rmse = mean_squared_error(y_test, preds_test)
        test_r2 = r2_score(y_test, preds_test)
        print(f"Test RMSE (tuned): {test_rmse:.4f}  Test R² (tuned): {test_r2:.4f}")

        # Save tuned model
        model_fname = f"models/best_tuned_{label}_{target}.pkl"
        with open(model_fname, "wb") as f:
            pickle.dump(tuned, f)
        print("Saved tuned model:", model_fname)

        # 5-fold CV R2 plot (using tuned model)
        cv_plot_path = f"cv_plots/cv_r2_{label}_{target}.png"
        try:
            cv_scores = plot_cv_r2(tuned, X_all, y_all, f"5-Fold CV R² ({label} - {target})", cv_plot_path)
            print("CV R² scores:", cv_scores, "mean:", np.mean(cv_scores))
        except Exception as e:
            print("Error computing cross-val scores:", e)
            cv_scores = []

        # Feature importance / SHAP plot
        imp_plot_path = f"importance_plots/importance_{label}_{target}.png"
        try:
            imp_df = compute_and_plot_importance(tuned, preprocessor, X_all, y_all, imp_plot_path, top_n=25, model_name=f"{label}_{target}")
            print("Saved importance plot:", imp_plot_path)
        except Exception as e:
            print("Importance computation failed:", e)

        # Save predictions on full dataset using tuned model
        preds_full = tuned.predict(X_all)
        df[f"pred_{label}_{target}"] = preds_full

        # Record summary
        summary_rows.append({
            "pipeline": label,
            "target": target,
            "base_model": best_base_name,
            "test_rmse": test_rmse,
            "test_r2": test_r2,
            "cv_mean_r2": float(np.mean(cv_scores)) if len(cv_scores)>0 else np.nan,
            "model_file": model_fname
        })

# -------------------------
# 7) Save full predictions to Excel
# -------------------------
out_pred_file = "predictions_full_dataset.xlsx"
df.to_excel(out_pred_file, index=False)
print("\nSaved full-dataset predictions to:", out_pred_file)
files.download(out_pred_file)

# -------------------------
# 8) Save summary CSV
# -------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv("tuning_summary.csv", index=False)
print("Saved tuning summary to tuning_summary.csv")
files.download("tuning_summary.csv")

# -------------------------
# 9) Generate Streamlit app file (streamlit_app.py)
# -------------------------
streamlit_code = r'''
import streamlit as st
import pandas as pd
import pickle
import os

st.title("Predict Li/Co/Mn/Ni - Tuned Models")
st.write("Upload a CSV with the input columns (same names as in training).")

uploaded = st.file_uploader("Upload CSV", type=["csv"])
if uploaded:
    df = pd.read_csv(uploaded)
    st.write("Preview:")
    st.dataframe(df.head())

    # Load models
    models = {}
    for label in ["withcat", "nocat"]:
        for metal in ["Li","Co","Mn","Ni"]:
            fname = f"models/best_tuned_{label}_{metal}.pkl"
            if os.path.exists(fname):
                models[f"{label}_{metal}"] = pickle.load(open(fname,"rb"))

    st.write("Loaded models:", list(models.keys()))

    if st.button("Run predictions"):
        # Decide pipeline type by presence of categorical columns
        need_cat = "Leaching agent" in df.columns and "Type of reducing agent" in df.columns

        results = df.copy()
        for key, model in models.items():
            # skip models that don't match input features (withcat vs nocat)
            if ("withcat" in key and not need_cat) or ("nocat" in key and need_cat and False):
                # only run nocat if we only have numeric inputs; run both otherwise
                pass
            try:
                preds = model.predict(df)
                results[f"pred_{key}"] = preds
            except Exception as e:
                st.write("Failed to predict with model", key, ":", e)

        st.write("Predictions sample:")
        st.dataframe(results.head())

        outname = "predictions_from_streamlit.xlsx"
        results.to_excel(outname, index=False)
        st.write(f"Saved predictions to {outname}")
        with open(outname, "rb") as f:
            st.download_button("Download predictions", f, file_name=outname)
'''

with open("streamlit_app.py", "w") as f:
    f.write(streamlit_code)

print("Wrote streamlit_app.py. To run locally: `streamlit run streamlit_app.py`")

# Done
print("\nALL DONE. Generated artifacts:")
print(" - models/ (saved tuned models)")
print(" - cv_plots/ (cv R2 plots)")
print(" - importance_plots/ (SHAP / importance plots)")
print(" - predictions_full_dataset.xlsx")
print(" - tuning_summary.csv")
print(" - streamlit_app.py")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.4 MB/s eta 0:00:00
